In [6]:
from __future__ import division, print_function
import GPy
from bayesian_linear_regression import BayesianLinearRegression as BLR 
import numpy as np

# %matplotlib inline  
# %matplotlib qt
from matplotlib import pyplot as plt

In [7]:
# Define the underlying function
def get_samples(x_query):
    y_query = np.zeros_like(x_query)
    
    mask_0to1 = x_query < 1
    mask_1to2 = (x_query > 1) & (x_query < 2)
    mask_2to3 = (x_query > 2) & (x_query < 3)
    
    n_0to1 = mask_0to1.sum()
    n_1to2 = mask_1to2.sum()
    n_2to3 = mask_2to3.sum()
    
    y_query[mask_0to1] = 0.1 * np.random.randn(n_0to1)
    y_query[mask_1to2] = 1 * np.random.randn(n_1to2)
    y_query[mask_2to3] = 2.0 * np.random.randn(n_2to3)
    
    return y_query

In [8]:
# Generate dataset
n = 200
x_train = np.sort(np.random.uniform(0, 3, 3*n)).reshape(-1, 1)
y_train = get_samples(x_train)
f, a = plt.subplots(1, 1)
a.plot(x_train, y_train, '.', color='gray')
a.set_ylim(-6., 6.)
a.grid()
a.set_xlabel('x')
a.set_ylabel('g')
a.set_title('Training Dataset')
# plt.show()

Text(0.5, 1.0, 'Training Dataset')

In [9]:
# Optimise GP hyperparameters using training data from whole domain
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(x_train, y_train, kernel)
m.rbf.lengthscale.constrain_bounded(0.1, 10)
m.optimize_restarts(num_restarts=2)

# Set BLR Prior using data from the whole domain
blr_prior_strength = 10  # lower = faster adaptation, higher = smoother adaptation
mdl = BLR()
x_blr_train = np.ones_like(x_train)
mdl.set_prior_from_data(x_blr_train, y_train, n=blr_prior_strength)
mdl.set_posterior_to_prior() # can use mdl.fit_posterior(X, y) to fit to new data. mdl.fit_posterior does not update the prior.

# Generate query points
x_query_0to1 = np.linspace(0.1, 0.9, 100).reshape(-1, 1)
x_query_1to2 = np.linspace(1.1, 1.9, 100).reshape(-1, 1)
x_query_2to3 = np.linspace(2.1, 2.9, 100).reshape(-1, 1)

# Plot the results using the GP model
f_gp, a_gp = plt.subplots(1, 1)
f_blr, a_blr = plt.subplots(1, 1)
a = [a_gp, a_blr]

for ax in a:
    ax.plot(x_train, y_train, '.', color='gray')
    ax.set_ylim(-6., 6.)
    ax.set_xlabel('x')
    ax.set_ylabel('g')
    ax.grid()
for x_q in [x_query_0to1, x_query_1to2, x_query_2to3]:
    
    # Get local data from the query range of 'x'
    local_mask = (x_train > min(x_q)) & (x_train < max(x_q))
    x_local = x_train[local_mask].reshape(-1,1)
    y_local = y_train[local_mask].reshape(-1,1)
    
    # Fit local GP model
    m.set_XY(X=x_local, Y=y_local)
    
    # Fit local BLR model 
    mdl.fit_posterior(X=x_local, Y=y_local)
    
    # Make predictions using the local GP model
    mu_gp, var_gp = m.predict(x_q)
    stdev_gp = np.sqrt(var_gp)
    
    # Make predictions using the local BLR model
    mu_blr, stdev_blr = mdl.pred_posterior(np.ones_like(x_q))
    print(stdev_blr.mean())
    # Plot predictions from the local GP model
    l, = a[0].plot(x_q, mu_gp)
    a[0].fill(np.concatenate([x_q, x_q[::-1]]),
              np.concatenate([mu_gp + stdev_gp, 
                              mu_gp[::-1] - stdev_gp[::-1]]),
              alpha=0.5, color=l.get_color())
    
    # Plot predictions from the local BLR model
    l, = a[1].plot(x_q, mu_blr)
    a[1].fill(np.concatenate([x_q, x_q[::-1]]), 
              np.concatenate([(mu_blr + stdev_blr),
                              (mu_blr - stdev_blr)[::-1]]), 
              alpha=0.5, color=l.get_color())
a[0].set_title('Local Gaussian Process')
a[1].set_title('Local Bayesian Linear Regression')

reconstraining parameters GP_regression.rbf.lengthscale


Optimization restart 1/2, f = 992.7081553977034
Optimization restart 2/2, f = 990.1917978831816
0.3252493642215592
0.9852045605820193
1.8873984720463322


Text(0.5, 1.0, 'Local Bayesian Linear Regression')

In [ ]:
# m.rbf.variance
# m.rbf.lengthscale
# m.Gaussian_noise.variance